In [ ]:
# Install required libraries
!pip install -q transformers datasets peft accelerate bitsandbytes sacremoses fsspec==2025.3.2

# Check that a GPU is available (for faster training and 8-bit quantization)
import torch
print("Running on GPU:", torch.cuda.is_available())

from google.colab import drive
drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/biogpt_lora_model"  # You can rename as needed



Running on GPU: True
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
from datasets import Dataset

# Load the JSONL file manually into a list of dicts
jsonl_path = "/content/mimic_cxr_finetune.jsonl"
data = []
with open(jsonl_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Create a HuggingFace Dataset from the list
raw_dataset = Dataset.from_list(data)

# Split into train and validation sets (e.g., 99% train, 1% eval)
split_datasets = raw_dataset.train_test_split(test_size=0.01, seed=42)
train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]

# Print example
print(train_dataset[0])
print(f"Train samples: {len(train_dataset)}; Validation samples: {len(eval_dataset)}")


{'prompt': 'Atelectasis: uncertain; Cardiomegaly: 1; Consolidation: uncertain; Edema: 1; Pleural Effusion: 1. \nFindings:', 'completion': " ET and NG tube are nominal in position.\n \n Cardiomegaly could be slightly more pronounced.\n \n Again seen are bilateral right greater left effusions with underlying collapse\n and/or consolidation.\n \n Also again seen is CHF, with upper zone redistribution, vascular plethora and\n probable vascular blurring.  On today's exam, there is increased obscuration\n of the right hemidiaphragm, which could reflect increased collapse/\n consolidation at the right base.  Biapical pleural scarring, with surface\n calcification again noted.  There is also nearby vascular calcification.\nImpression: Question interval increase in degree of cardiomegaly.\n \n CHF and bilateral effusions again seen.\n \n Bibasilar collapse and/or consolidation.  This is likely worse on the right,\n where the right hemidiaphragm is now obscured.", 'study_id': 52711754, 'subject_

In [ ]:
from transformers import AutoTokenizer

# Choose the model checkpoint and tokenizer.
model_name = "microsoft/biogpt"  # BioGPT (biomedical GPT-2 like model)
# If BioGPT is not suitable or unavailable, you could use a general model:
# model_name = "EleutherAI/gpt-neo-1.3B"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
# For GPT models, the pad_token might be missing by default. We set it to the EOS token to avoid errors.
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Define the maximum sequence length
MAX_LEN = 512

def preprocess_function(example):
    prompt = example["prompt"]
    completion = example["completion"]
    # Combine prompt and completion with a newline separator.
    # (The newline helps the model distinguish prompt from report text.)
    full_text = prompt + "\n" + completion

    # Tokenize the combined text. We use truncation and padding to MAX_LEN.
    tokens = tokenizer(full_text, max_length=MAX_LEN, truncation=True, padding="max_length")
    input_ids = tokens["input_ids"]
    attention_mask = tokens["attention_mask"]

    # Prepare labels: copy of input_ids
    labels = input_ids.copy()
    # Determine prompt token length (including the trailing newline as part of prompt).
    # We tokenize the prompt with a newline to get the token count for prompt segment.
    prompt_tokens = tokenizer(prompt + "\n", add_special_tokens=False).input_ids
    prompt_length = len(prompt_tokens)
    if prompt_length > MAX_LEN:
        prompt_length = MAX_LEN  # edge case: if prompt itself is longer than MAX_LEN

    # Mask out the prompt tokens in the labels
    for i in range(min(prompt_length, len(labels))):
        labels[i] = -100  # -100 will mask this token from loss computation

    # Also mask padding tokens in the labels
    # (If pad_token_id equals eos_token_id, we still don’t want to compute loss on padding.)
    pad_token_id = tokenizer.pad_token_id
    labels = [(-100 if token_id == pad_token_id else token_id) for token_id in labels]

    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

# Apply the preprocessing to the training and validation datasets
tokenized_train = train_dataset.map(preprocess_function, batched=False, remove_columns=train_dataset.column_names)
tokenized_eval = eval_dataset.map(preprocess_function, batched=False, remove_columns=eval_dataset.column_names)
print("Tokenization sample:", tokenized_train[0]["input_ids"][:10], "...\nCorresponding labels:", tokenized_train[0]["labels"][:10], "...")


Map:   0%|          | 0/59199 [00:00<?, ? examples/s]

Map:   0%|          | 0/598 [00:00<?, ? examples/s]

Tokenization sample: [2, 5619, 12805, 38946, 20, 6625, 44, 15489, 17921, 20] ...
Corresponding labels: [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100] ...


In [ ]:
# !pip uninstall -y bitsandbytes
# !pip install git+https://github.com/jllllll/bitsandbytes-wheels.git@main

!pip install -U bitsandbytes


In [ ]:
!pip install -U transformers accelerate peft


In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

model_name = "microsoft/biogpt"

# Load the pre-trained base model in 8-bit mode.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,    # load weights in int8 precision
    device_map="auto"     # let accelerate auto-distribute the model on available devices (GPU)
)

# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,
#     llm_int8_threshold=6.0,
#     llm_int8_has_fp16_weight=True
# )

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=bnb_config,
#     device_map="auto"
# )

# Prepare model for 8-bit training (this enables gradient checkpointing and other tweaks for int8).
model = prepare_model_for_kbit_training(model)

# Define LoRA configuration for causal LM
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # LoRA will be applied to Query and Value projection matrices
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False  # we are setting up for training
)

# Attach LoRA adapters to the model
model = get_peft_model(model, lora_config)

# (Optional) Print the number of trainable parameters to verify LoRA is applied
trainable_params = 0
total_params = 0
for param in model.parameters():
    total_params += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
print(f"Trainable params: {trainable_params} (~{100 * trainable_params/ total_params:.2f}% of total)")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Trainable params: 786432 (~0.23% of total)


In [ ]:
# # Testing for small dataset
# small_train_dataset = tokenized_train.shuffle(seed=42).select(range(2000))
# small_eval_dataset = tokenized_eval.shuffle(seed=42).select(range(100))


In [ ]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="radiology_report_lora",   # output directory for model checkpoints
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,        # accumulate gradients to simulate batch_size = 16
    num_train_epochs=2,                   # train for 2 epochs (can adjust to 1 or more as needed)
    learning_rate=2e-4,
    fp16=True,                            # enable mixed precision training
    logging_steps=500,                    # log training metrics every 500 steps
    eval_strategy="epoch",          # evaluate on the validation set at end of each epoch
    save_strategy="epoch",                # save model at end of each epoch
    save_total_limit=1,                   # only keep the most recent checkpoint
    report_to="none"                      # no integrated logging to WandB or others
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    args=training_args,
)

# Start training
trainer.train()

save_path = "/content/drive/MyDrive/biogpt_lora_model"

# Save the LoRA-adapted model (this saves the LoRA adapters and configuration).
trainer.model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model saved to: {save_path}")


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization


Epoch,Training Loss,Validation Loss
1,1.854100,1.741544


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
1,1.854100,1.741544
2,1.799400,1.690355


Model saved to: /content/drive/MyDrive/biogpt_lora_model
Model saved to: /content/drive/MyDrive/biogpt_lora_model


In [ ]:
# from peft import PeftModel

# # Reload the base model and apply the LoRA adapters (if starting fresh, e.g., in a new session):
# base_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)
# model = PeftModel.from_pretrained(base_model, "radiology_report_lora")
# model.eval()  # set to evaluation mode

# # Ensure tokenizer is loaded (if not already from previous steps)
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
# if tokenizer.pad_token_id is None:
#     tokenizer.pad_token_id = tokenizer.eos_token_id

# # Define a sample prompt (e.g., simulated diagnosis confidence scores for a chest X-ray)
# sample_prompt = "Cardiomegaly: 0; Consolidation: 1; Atelectasis: 0"

# # Tokenize the prompt and generate a report
# input_ids = tokenizer(sample_prompt, return_tensors="pt").input_ids.to(model.device)
# # Generate the completion (report). Limit the length of generation to avoid run-on.
# # We'll use greedy decoding for simplicity; for more varied text, one could use beam search or sampling.
# outputs = model.generate(input_ids, max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
# generated_ids = outputs[0][input_ids.shape[-1]:]  # slice out the generated portion (exclude prompt tokens)
# generated_report = tokenizer.decode(generated_ids, skip_special_tokens=True)

# print(f"Prompt:\n{sample_prompt}\n")
# print(f"Generated Report:\n{generated_report}")

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Paths
base_model_name = "microsoft/biogpt"
lora_path = "/content/drive/MyDrive/biogpt_lora_model"

# Load base model in float16 instead of 8-bit
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, lora_path)
model.eval()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Example prompt
sample_prompt = "Cardiomegaly: 0; Consolidation: 1; Atelectasis: 0"

# Tokenize and generate
input_ids = tokenizer(sample_prompt, return_tensors="pt").input_ids.to(model.device)
outputs = model.generate(
    input_ids,
    max_new_tokens=200,
    pad_token_id=tokenizer.eos_token_id
)

# Decode
generated_ids = outputs[0][input_ids.shape[-1]:]
generated_report = tokenizer.decode(generated_ids, skip_special_tokens=True)

# Output
print(f"Prompt:\n{sample_prompt}\n")
print(f"Generated Report:\n{generated_report}")




The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Prompt:
Cardiomegaly: 0; Consolidation: 1; Atelectasis: 0

Generated Report:
. There is no focal consolidation, pleural effusion, or pneumothorax. The cardiac silhouette is top normal in size. The mediastinal and hilar contours are normal. The pulmonary vasculature is normal. Impression: No focal consolidation to suggest pneumonia. No evidence of acute cardiopulmonary process. No cardiomegaly. No pulmonary vascular congestion. No focal consolidation to suggest pneumonia. No large pleural effusion. No acute cardiopulmonary process. No evidence of cardiomegaly. No evidence of pulmonary vascular congestion. No evidence of acute cardiopulmonary process. No evidence of cardiomegaly. No evidence of pulmonary vascular congestion. No evidence of acute cardiopulmonary process. No evidence of cardiomegaly. No evidence of pulmonary vascular congestion. No evidence of pulmonary vascular engorgement. No evidence of acute cardiopulmonary process. No evidence of cardiomegaly. No evidence of pulmonary

In [ ]:
# from peft import PeftModel

# # Reload the base model and apply the LoRA adapters (if starting fresh, e.g., in a new session):
# base_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)
# model = PeftModel.from_pretrained(base_model, "radiology_report_lora")
# model.eval()  # set to evaluation mode

# # Ensure tokenizer is loaded (if not already from previous steps)
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
# if tokenizer.pad_token_id is None:
#     tokenizer.pad_token_id = tokenizer.eos_token_id

# # Define a sample prompt (e.g., simulated diagnosis confidence scores for a chest X-ray)
# sample_prompt = "Cardiomegaly: 0; Consolidation: 1; Atelectasis: 0"

# # Tokenize the prompt and generate a report
# input_ids = tokenizer(sample_prompt, return_tensors="pt").input_ids.to(model.device)
# # Generate the completion (report). Limit the length of generation to avoid run-on.
# # We'll use greedy decoding for simplicity; for more varied text, one could use beam search or sampling.
# outputs = model.generate(input_ids, max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
# generated_ids = outputs[0][input_ids.shape[-1]:]  # slice out the generated portion (exclude prompt tokens)
# generated_report = tokenizer.decode(generated_ids, skip_special_tokens=True)

# print(f"Prompt:\n{sample_prompt}\n")
# print(f"Generated Report:\n{generated_report}")

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# Paths
base_model_name = "microsoft/biogpt"
lora_path = "/content/drive/MyDrive/biogpt_lora_model"

# Load base model in float16 instead of 8-bit
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, lora_path)
model.eval()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Example prompt
sample_prompt = "Cardiomegaly: 1; Consolidation: 1"

# Tokenize and generate
input_ids = tokenizer(sample_prompt, return_tensors="pt").input_ids.to(model.device)
outputs = model.generate(
    input_ids,
    max_new_tokens=200,
    pad_token_id=tokenizer.eos_token_id
)

# Decode
generated_ids = outputs[0][input_ids.shape[-1]:]
generated_report = tokenizer.decode(generated_ids, skip_special_tokens=True)

# Output
print(f"Prompt:\n{sample_prompt}\n")
print(f"Generated Report:\n{generated_report}")


